## BBH

In [58]:
#from ler.gw_source_population import merger_rate_density_bbh_popI_II_oguri2018
import numpy as np
import matplotlib.pyplot as plt
# interpolate
from scipy.interpolate import interp1d
from scipy.integrate import trapz, simps, quad
from gwsnr import GWSNR
from ler.rates import LeR
from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

In [59]:
# zs to dl
zs_arr = np.geomspace(0.0001, 10, 10000)
dl_arr = cosmo.luminosity_distance(zs_arr).value
zs_to_dl = interp1d(zs_arr, dl_arr, kind='cubic')
dl_to_zs = interp1d(dl_arr, zs_arr, kind='cubic')

## Horizon distance

In [60]:
snr = GWSNR(gwsnr_verbose=False, ifos=['L1'])

Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/partialSNR_dict_0.pickle


In [61]:
snr.detector_horizon(mass_1=20, mass_2=20)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.55s/it]

dl_eff 123.76356521495073
optimal_snr_unscaled [215.0957101]


{'L1': array([3327.62649311]), 'net': array([3327.62649311])}

## Merger rate density

In [62]:
def merger_rate_density_bbh_popI_II_oguri2018(
        zs, R0=29.3 * 1e-9, b2=1.6, b3=2.0, b4=30,
    ):
    """
    Function to compute the merger rate density (PopI/PopII). Reference: Oguri et al. (2018). The output is in detector frame and is unnormalized.

    Parameters
    ----------
    zs : `float` or `numpy.ndarray` (nD array of floats)
        Source redshifts
    R0 : `float`
        local merger rate density at low redshift
        default: 105.5*1e-9 Mpc^-3 yr^-1
    b2 : `float`
        Fitting paramters
        default: 1.6
    b3 : `float`
        Fitting paramters
        default: 2.0
    b4 : `float`
        Fitting paramters
        default: 30

    Returns
    ----------
    rate_density : `float` or `numpy.ndarray` (nD array of floats)
        merger rate density

    Examples
    ----------
    >>> from ler.gw_source_population import merger_rate_density_bbh_popI_II_oguri2018
    >>> rate_density = merger_rate_density_bbh_popI_II_oguri2018(zs=0.1)
    """
    
    # rate_density
    return R0 * (b4 + 1) * np.exp(b2 * zs) / (b4 + np.exp(b3 * zs))

In [63]:
def snr_operator(zs):

    dl_hor = 3327.62649311
    dl = cosmo.luminosity_distance(zs).value
    result = np.zeros_like(zs)
    result[dl < dl_hor] = 1

    return result


In [64]:
snr_operator(0.5)

array(1.)

## Unlensed Rate

\begin{equation}
\begin{aligned}
\mathcal{R}^U = \int_{0}^{z_{\text{max}}} \frac{R(z_s)}{1+z_s}\frac{dV_c}{dz_s} \Theta(z_{\text{hor}}-z_s) dz_s
\end{aligned}
\end{equation}

Or,

\begin{equation}
\begin{aligned}
\mathcal{R}^U = \int_{0}^{z_{\text{hor}}} \frac{R(z_s)}{1+z_s}\frac{dV_c}{dz_s} dz_s
\end{aligned}
\end{equation}

### Integration with `quad` function

In [65]:
# intergrand function
def unlensed_integrand(zs):

    merger_rate = merger_rate_density_bbh_popI_II_oguri2018(zs=zs)
    dvc_dz = cosmo.differential_comoving_volume(zs).value * 4 * np.pi
    snr_op = snr_operator(zs=zs)
    
    return merger_rate/(1+zs) * dvc_dz * snr_op

In [66]:
# integral
integral_unlensed = quad(unlensed_integrand, 0, 10)[0]

In [67]:
integral_unlensed

1543.5254979471417

### Integration with `np.trapz` function

In [68]:
# intergrand function
def unlensed_integrand(zs):

    merger_rate = merger_rate_density_bbh_popI_II_oguri2018(zs=zs)
    dvc_dz = cosmo.differential_comoving_volume(zs).value * 4 * np.pi
    
    result = merger_rate/(1+zs) * dvc_dz * snr_operator(zs)

    return result 

In [69]:
zs = np.geomspace(0.0001, 10, 10000)
unlensed_integrand_arr = np.array([unlensed_integrand(z) for z in zs])
integral_unlensed = np.trapz(unlensed_integrand_arr, zs)

In [70]:
integral_unlensed

1545.0588528302608

## Lensed rate

\begin{equation}
\begin{aligned}
\mathcal{R}^L = \int_{0}^{z_{\text{max}}} \frac{R(z_s)}{1+z_s}\frac{dV_c}{dz_s} \Theta(z_{\text{hor}}-z_s) \tau(z_s) P(\mu) dz_s
\end{aligned}
\end{equation}

- $P(\mu) = 8 \mu^{-3}$



In [81]:
# redshift to comoving distance
cosmo.comoving_distance(0.1)

<Quantity 418.45448763 Mpc>

In [82]:
def optical_depth_SIS_haris(zs):
    """
    Function to compute the strong lensing optical depth (SIS). \n
    LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) was used to derive the following equation. This is the analytic version of optical depth from z=0 to z=zs.

    Parameters
    ----------
    zs : `float`
        source redshifts

    Returns
    -------
    tau : `float`
        strong lensing optical depth
    """

    # z to luminosity_distance (luminosity_distance) conversion
    Dc = cosmo.comoving_distance(zs).value * 1e-3  # 1e-3 converts Mpc to Gpc

    return (Dc / 62.2) ** 3  # 62.2 is the critical density in Gpc

In [83]:
def magnification_function(mu):
    return 8 / mu**3

In [84]:
cosmo.luminosity_distance(0.1)

<Quantity 460.29993639 Mpc>

In [85]:
def snr_operator_lensed(zs, mu):

    dl_hor = 416.49499405
    # redshift to luminosity_distance conversion
    dl = cosmo.luminosity_distance(zs).value # Mpc
    dl_eff = dl / np.sqrt(mu) # effective luminosity distance
    result = np.zeros_like(dl_eff)
    result[dl_eff < dl_hor] = 1
    return result

In [86]:
# intergrand function
# def lensed_integrand(zs):

#     merger_rate = merger_rate_density_bbh_popI_II_oguri2018(zs=zs)
#     dvc_dz = cosmo.differential_comoving_volume(zs).value * 4 * np.pi
#     tau = optical_depth_SIS_haris(zs)

#     # integration over only mu dependent part
#     integrand_fn = lambda mu:  snr_operator_lensed(zs=zs, mu=mu)  * magnification_function(mu)
    
#     result = quad(integrand_fn, 2, 1e4)[0]

#     return result * merger_rate/(1+zs) * dvc_dz* tau

In [87]:
# intergrand function
def lensed_integrand(zs):

    merger_rate = merger_rate_density_bbh_popI_II_oguri2018(zs=zs)
    dvc_dz = cosmo.differential_comoving_volume(zs).value * 4 * np.pi
    tau = optical_depth_SIS_haris(zs)

    # integration over only mu dependent part
    integrand_fn = lambda mu:  merger_rate/(1+zs) * dvc_dz* tau * snr_operator_lensed(zs=zs, mu=mu)  * magnification_function(mu)

    mu = np.geomspace(2, 1e4, 1000)

    integrand_arr = integrand_fn(mu)
    
    result = np.trapz(integrand_arr, x=mu)

    return result 

In [88]:
# integral
zs = np.geomspace(0.0001, 10, 10000)
lensed_integrand_arr = []
for z in zs:
    lensed_integrand_arr.append(lensed_integrand(z))
lensed_integrand_arr = np.array(lensed_integrand_arr)
integral_lensed = np.trapz(lensed_integrand_arr, zs)

In [89]:
integral_lensed

0.0003891721823019238

In [90]:
integral_unlensed/integral_lensed

3970116.372890157

## Unlensed monte-carlo integration

\begin{equation}
\begin{aligned}
\mathcal{R}^U = N \left<\Theta(z_{\text{hor}}-z_s)\right>_{z_s\in P(z_s)} 
\end{aligned}
\end{equation}

- $P(z_s) = \frac{1}{N} \frac{R(z_s)}{1+z_s} \frac{dV_c}{dz_s}$

- $N=\int \frac{R(z_s)}{1+z_s} \frac{dV_c}{dz_s} dz_s$

In [ ]:
# find normalization factor
buffer_fn = lambda zs: merger_rate_density_bbh_popI_II_oguri2018(zs=zs)/(1+zs) * cosmo.differential_comoving_volume(zs).value * 4 * np.pi

In [ ]:
# intergrand function
def unlensed_integrand(zs):

    merger_rate = merger_rate_density_bbh_popI_II_oguri2018(zs=zs)
    dvc_dz = cosmo.differential_comoving_volume(zs).value * 4 * np.pi
    snr_op = snr_operator(zs=zs)
    
    return merger_rate/(1+zs) * dvc_dz * snr_op